In [1]:
!pip install uv -q

In [2]:
!uv pip install "unstructured[md]" nltk langchain_google_genai pypdf langchain_community langchain_core langchain streamlit -q

In [3]:
from langchain_community.document_loaders import UnstructuredMarkdownLoader
from langchain_core.documents import Document

markdown_path = "/home/pratham/Desktop/projects/codebasics/ds-rpc-01/resources/data/engineering/engineering_master_doc.md"
loader = UnstructuredMarkdownLoader(markdown_path)

engineering_data = loader.load()
len(engineering_data)
# assert len(data) == 1
# assert isinstance(data[0], Document)
# readme_content = data[0].page_content
# print(readme_content[:250])

1

In [4]:
page_content = engineering_data[0].page_content
print(page_content[:200])

FinSolve Technologies Engineering Document

1. Introduction

1.1 Company Overview

FinSolve Technologies is a leading FinTech company headquartered in Bangalore, India, with operations across North Am


In [5]:
from dotenv import load_dotenv
import os
load_dotenv()  # take environment variables

GOOGLE_API_KEY = os.environ['GOOGLE_API_KEY']

In [6]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings

embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001", google_api_key = GOOGLE_API_KEY)
# embeddings.embed_query("What's our Q1 revenue?")

In [7]:
from langchain_google_genai import GoogleGenerativeAI
llm = GoogleGenerativeAI(model="gemma-3n-e4b-it", google_api_key = GOOGLE_API_KEY)
llm = GoogleGenerativeAI(model="gemini-2.0-flash-lite", google_api_key =GOOGLE_API_KEY)



In [8]:
!uv pip install -qU langchain_community faiss-cpu

In [9]:
from langchain_community.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=200)

split_text = text_splitter.split_documents(engineering_data)

vector_store = FAISS.from_documents(split_text, embedding=embeddings)

vector_store.save_local("faiss_index")



In [10]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser


prompt = PromptTemplate(template='you are assistant having knowledge of the engineering dept as per question extract the relevent chunks and build answer if you did not find the answer in context say i do not know the asnwer {question} \n {context}', input_variables=["context", "question"])

chain = prompt | llm | StrOutputParser()

In [11]:
new_db = FAISS.load_local("faiss_index", embeddings, allow_dangerous_deserialization=True)
user_question = 'what is name of the company?'
docs = new_db.similarity_search(user_question)
response = chain.invoke({"context": docs, "question": user_question})
print(response)

The name of the company is FinSolve Technologies.


In [12]:
from langchain_community.document_loaders import DirectoryLoader
loader = DirectoryLoader('../resources/data')
docs = loader.load()
len(docs)

10

In [13]:
docs

[Document(metadata={'source': '../resources/data/engineering/engineering_master_doc.md'}, page_content='FinSolve Technologies Engineering Document\n\n1. Introduction\n\n1.1 Company Overview\n\nFinSolve Technologies is a leading FinTech company headquartered in Bangalore, India, with operations across North America, Europe, and Asia-Pacific. Founded in 2018, FinSolve provides innovative financial solutions, including digital banking, payment processing, wealth management, and enterprise financial analytics, serving over 2 million individual users and 10,000 businesses globally.\n\n1.2 Purpose\n\nThis engineering document outlines the technical architecture, development processes, and operational guidelines for FinSolve\'s product ecosystem. It serves as a comprehensive guide for engineering teams, stakeholders, and partners to ensure alignment with FinSolve\'s mission: "To empower financial freedom through secure, scalable, and innovative technology solutions."\n\n1.3 Scope\n\nThis docu

In [14]:
import re

# Update the metadata of documents based on the source file path
department_list = ['engineering', 'finance', 'hr', 'marketing']
updated_docs = []
for doc in docs:
    source = doc.metadata.get('source', '')
    match = re.search(r'/data/([^/]+)/', source)
    if match:
        dept = match.group(1)
        if dept == 'general':
            # For general documents, give access to all departments
            doc.metadata['department'] = department_list
        else:
            # For specific department documents
            doc.metadata['department'] = dept
        updated_docs.append(doc)
docs = updated_docs
docs

[Document(metadata={'source': '../resources/data/engineering/engineering_master_doc.md', 'department': 'engineering'}, page_content='FinSolve Technologies Engineering Document\n\n1. Introduction\n\n1.1 Company Overview\n\nFinSolve Technologies is a leading FinTech company headquartered in Bangalore, India, with operations across North America, Europe, and Asia-Pacific. Founded in 2018, FinSolve provides innovative financial solutions, including digital banking, payment processing, wealth management, and enterprise financial analytics, serving over 2 million individual users and 10,000 businesses globally.\n\n1.2 Purpose\n\nThis engineering document outlines the technical architecture, development processes, and operational guidelines for FinSolve\'s product ecosystem. It serves as a comprehensive guide for engineering teams, stakeholders, and partners to ensure alignment with FinSolve\'s mission: "To empower financial freedom through secure, scalable, and innovative technology solution

In [15]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)

split_text = text_splitter.split_documents(docs)

vector_store = FAISS.from_documents(split_text, embedding=embeddings)

vector_store.save_local("faiss_index")

In [16]:
docs

[Document(metadata={'source': '../resources/data/engineering/engineering_master_doc.md', 'department': 'engineering'}, page_content='FinSolve Technologies Engineering Document\n\n1. Introduction\n\n1.1 Company Overview\n\nFinSolve Technologies is a leading FinTech company headquartered in Bangalore, India, with operations across North America, Europe, and Asia-Pacific. Founded in 2018, FinSolve provides innovative financial solutions, including digital banking, payment processing, wealth management, and enterprise financial analytics, serving over 2 million individual users and 10,000 businesses globally.\n\n1.2 Purpose\n\nThis engineering document outlines the technical architecture, development processes, and operational guidelines for FinSolve\'s product ecosystem. It serves as a comprehensive guide for engineering teams, stakeholders, and partners to ensure alignment with FinSolve\'s mission: "To empower financial freedom through secure, scalable, and innovative technology solution

In [17]:
# # Step 1: Run similarity search
# results = vector_store.similarity_search(
#     query="what is company overview?",
#     k=10  # Fetch more so filtering is meaningful
# )

# # Step 2: Enhanced filtering to handle both string and list departments
# filtered_results = []
# for doc in results:
#     department = doc.metadata.get("department", "")
#     source = doc.metadata.get("source", "")
    
#     # Check if document should be included based on department
#     should_include = False
    
#     # Handle case where department is a string
#     if isinstance(department, str):
#         if ("general" in department.lower() or 
#             department.lower() == "marketing"):
#             should_include = True
    
#     # Handle case where department is a list
#     elif isinstance(department, list):
#         for dept in department:
#             if ("general" in dept.lower() or 
#                 dept.lower() == "marketing"):
#                 should_include = True
#                 break
    
#     # Also check source field for "general"
#     if "general" in source.lower():
#         should_include = True
    
#     if should_include:
#         filtered_results.append(doc)

# # Step 3: Print results
# for doc in filtered_results:
#     print(f"* {doc.page_content} [{doc.metadata}]")

# # Optional: Print summary of filtering
# print(f"\nFiltered {len(filtered_results)} documents from {len(results)} total results")

In [18]:
def answer(question, input_department):
    # Step 1: Use retriever with the actual question
    retriever = vector_store.as_retriever(search_type="mmr", search_kwargs={"k": 5})
    results = retriever.invoke(question)

    # Step 2: Enhanced filtering to handle both string and list departments
    filtered_results = []
    for doc in results:
        department = doc.metadata.get("department", "")
        source = doc.metadata.get("source", "")
        
        # Check if document should be included based on department
        should_include = False
        
        # Handle case where department is a list
        if isinstance(department, list):
            if "general" in department or input_department in department:
                should_include = True
        # Handle case where department is a string
        elif isinstance(department, str):
            if department.lower() == "general" or department.lower() == input_department.lower():
                should_include = True
        
        # Also check source field for department or general
        if "general" in source.lower() or input_department.lower() in source.lower():
            should_include = True
        
        if should_include:
            filtered_results.append(doc)

    if not filtered_results:
        return "I am sorry, I cannot answer the question as no relevant documents were found.", []

    context = filtered_results
    
    prompt = PromptTemplate(template='you are assistant having knowledge of the engineering dept as per question extract the relevent chunks and build answer if you did not find the answer in context say i do not know the answer {question} \n {context}', input_variables=["context", "question"])

    chain = prompt | llm | StrOutputParser()
    
    response = chain.invoke({"context": context, "question": question})
    
    return response, context

In [19]:
response, context = answer(question='what is company overview?',input_department='engineering')
print(response)

The company overview is:

Founded in 2016, FinSolve Technologies is a leading player in fintech with a presence across India and global markets. They are committed to ethical business, social responsibility, and fostering a culture of learning and growth.


In [20]:
response

'The company overview is:\n\nFounded in 2016, FinSolve Technologies is a leading player in fintech with a presence across India and global markets. They are committed to ethical business, social responsibility, and fostering a culture of learning and growth.'

In [21]:
context

[Document(id='f9cdf831-442f-4b9f-9690-acd2a7553851', metadata={'source': '../resources/data/general/employee_handbook.md', 'department': ['engineering', 'finance', 'hr', 'marketing']}, page_content='Employee Handbook\n\nTable of Contents\n\nWelcome & Introduction\n\nEmployee Onboarding & Benefits\n\nLeave Policies\n\nWork Hours & Attendance\n\nCode of Conduct & Workplace Behavior\n\nHealth & Safety\n\nCompensation & Payroll\n\nReimbursement Policies\n\nTraining & Development\n\nPerformance & Feedback\n\nPrivacy & Data Security\n\nExit Policy\n\nFAQs\n\nMiscellaneous\n\nWelcome & Introduction\n\nCompany Vision and Mission'),
 Document(id='3a370954-cb33-4c46-adc6-88950aab0c84', metadata={'source': '../resources/data/engineering/engineering_master_doc.md', 'department': 'engineering'}, page_content='1.2 Purpose\n\nThis engineering document outlines the technical architecture, development processes, and operational guidelines for FinSolve\'s product ecosystem. It serves as a comprehensive 

In [23]:
query = 'can you draw the barchart showing how many as per the department'
dept = 'hr'

In [ ]:
def answer(question, input_department):
    # Step 1: Use retriever with the actual question
    retriever = vector_store.as_retriever(search_type="mmr", search_kwargs={"k": 5})
    results = retriever.invoke(question)

    # Step 2: Enhanced filtering to handle both string and list departments
    filtered_results = []
    for doc in results:
        department = doc.metadata.get("department", "")
        source = doc.metadata.get("source", "")
        
        # Check if document should be included based on department
        should_include = False
        
        # Handle case where department is a list
        if isinstance(department, list):
            if "general" in department or input_department in department:
                should_include = True
        # Handle case where department is a string
        elif isinstance(department, str):
            if department.lower() == "general" or department.lower() == input_department.lower():
                should_include = True
        
        # Also check source field for department or general
        if "general" in source.lower() or input_department.lower() in source.lower():
            should_include = True
        
        if should_include:
            filtered_results.append(doc)

    if not filtered_results:
        return "I am sorry, I cannot answer the question as no relevant documents were found.", []

    context = filtered_results
    
    prompt = PromptTemplate(template='you are assistant having knowledge of the engineering dept as per question extract the relevent chunks and build answer if you did not find the answer in context say i do not know the answer {question} \n {context} if ', input_variables=["context", "question"])

    chain = prompt | llm 
    
    response = chain.invoke({"context": context, "question": question})
    
    return response, context

In [37]:
res, con = answer(question=query,input_department=dept)

In [38]:
res

'I am sorry, but I do not have the information to create a bar chart showing the number of employees per department.'